U.S. Census Bureau API with Python
======

## American Community Survey

----

*September 3, 2017*<br>
*@bd_econ*

Using the American Community Survey (ACS) to examine some demographic and economic trends at the U.S. county level.

List of variables from the [5-year ACS](https://www.census.gov/data/developers/data-sets/acs-5year.html) are found [here](https://api.census.gov/data/2015/acs5/variables.html).

The vincent example requires two topo.json files: [States](https://github.com/wrobstory/vincent_map_data/blob/master/us_states.topo.json) and [Counties](https://github.com/wrobstory/vincent_map_data/blob/master/us_counties.topo.json)

In [1]:
import requests
import pandas as pd

import config
key = config.census_key

In [2]:
base = 'https://api.census.gov/data/'
years = ['2015']#['2009', '2012', '2015']
variables = {'NAME':'Name',
             'B01001_001E': 'Population total',
             'B19013_001E': 'Real Median Income',}
v = ','.join(variables.keys())
c = '*'
s = '*'

In [6]:
requests.get(url).text

'<html><head><title>Error report</title></head><body><h1>HTTP Status 404 - /data/2015/acs5</h1></body></html>'

In [3]:
df = pd.DataFrame()
for y in years:
    url = '{}{}/acs5?get={}&for=county:{}&in=state:{}&key={}'.format(
        base, y, v, c, s, key)
    r = requests.get(url).json()
    dft = pd.DataFrame(r[1:], columns=r[0])
    dft['Year'] = y
    df = df.append(dft)
df = df.rename(columns=variables).set_index(
    ['Name', 'Year']).sort_index(level='Name')
df.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Map the results

Note: to make the example below work, you will first need to save [this](https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json) topo.json file in the same directory as the jupyter notebook.

In [ ]:
df['Real Median Income'] = df['Real Median Income'].astype(float)

df['FIPS'] = df['state'] + df['county']
df['FIPS'] = df['FIPS'].astype(int)
df['FIPS'] = df['FIPS'].map(lambda i: str(i).zfill(5))
# County FIP Codes that have changed:
df['FIPS'] = df['FIPS'].str.replace('46102', '46113')

In [ ]:
# For mapping results
import vincent
vincent.core.initialize_notebook()

geo_data = [{'name': 'counties',
             'url': 'geo/us_counties.topo.json',
             'feature': 'us_counties.geo'},            
            {'name': 'states',
             'url': 'geo/us_states.topo.json',
             'feature': 'us_states.geo'}
             ]

vis = vincent.Map(data=df, geo_data=geo_data, scale=1100,
                  projection='albersUsa', data_bind='Real Median Income',
                  data_key='FIPS', map_key={'counties': 'properties.FIPS'})

del vis.marks[1].properties.update
vis.marks[0].properties.enter.stroke.value = '#fff'
vis.marks[1].properties.enter.stroke.value = '#000000'
vis.scales['color'].domain = [0, 75000] # Adjust
vis.legend(title='Real Median Income')
vis.to_json('geo/vega.json')

vis.display()